In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Model, load_model
from keras.models import Model, load_model

In [11]:
datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10, 
    width_shift_range=0.15,
    height_shift_range=0.15,
    fill_mode='nearest')

In [12]:
train_generation = train_datagen.flow_from_directory(
    'train',
    target_size=(70,70),
    batch_size=64,
    class_mode='binary')
val_generator = datagen.flow_from_directory(
    'val',
    target_size=(70,70),
    batch_size=64,
    class_mode='binary')
test_generator = datagen.flow_from_directory(
    'test', 
    target_size=(70,70), 
    batch_size=64,
    class_mode='binary')

Found 17500 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.


In [13]:
batch_size = 64
nb_train = 17500
nb_valid = 3750
nb_test = 3750

In [14]:
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(70, 70, 3))
pretrained_model.trainable = False

In [ ]:
x =pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=pretrained_model.input, outputs=predictions)
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])
filepath="cat_dog_best_v3.0_only_HEI.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
calb = [checkpoint,learning_rate_reduction]
model.fit_generator(train_generation,
                    steps_per_epoch=nb_train//batch_size,
                    epochs=20,
                    validation_data=val_generator, 
                    validation_steps=nb_valid//batch_size, callbacks=calb)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20


In [ ]:
model.save('nn.h5')

In [ ]:
pretrained_model.trainable = False
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])
filepath="cat_dog_best_GOOD_BEST.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
calb = [checkpoint,learning_rate_reduction]
model.fit_generator(train_generation,
                    steps_per_epoch=nb_train//batch_size,
                    epochs=5,
                    validation_data=val_generator, 
                    validation_steps=nb_valid//batch_size, callbacks=calb)
model.save('Vgg1.h5')

In [ ]:
model.evaluate_generator(test_generator, nb_test/batch_size,pickle_safe=False)

In [ ]:
img = load_img('path', target_size=(70,70))
img = image.img_to_array(img)
img = img.astype('float32')
img = img/255
img = np.expand_dims(img, axis=0)
        

In [ ]:
model = Sequential()
model = load_model('Vgg1.h5')

In [ ]:
model.predict_on_batch(img)[0][0]
#If predict<0.55 - Its a cat